This is my final project.
The goal is to take a table of data on a lot of contrys and use it to predict a countrys co2 output at a given time. 


In [21]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
DF = pd.read_csv("energy.csv")
DF.head(7)


,Unnamed: 0,Country,Energy_type,Year,Energy_consumption,Energy_production,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP,CO2_emission
0,0,World,all_energy_types,1980,292.899790,296.337228,27770.910281,4.298127e+06,68.145921,10.547,4946.627130
1,1,World,coal,1980,78.656134,80.114194,27770.910281,4.298127e+06,68.145921,10.547,1409.790188
2,2,World,natural_gas,1980,53.865223,54.761046,27770.910281,4.298127e+06,68.145921,10.547,1081.593377
3,3,World,petroleum_n_other_liquids,1980,132.064019,133.111109,27770.910281,4.298127e+06,68.145921,10.547,2455.243565
4,4,World,nuclear,1980,7.575700,7.575700,27770.910281,4.298127e+06,68.145921,10.547,0.000000
5,5,World,renewables_n_other,1980,20.702344,20.775178,27770.910281,4.298127e+06,68.145921,10.547,0.000000
6,6,Afghanistan,all_energy_types,1980,0.026583,0.072561,NaN,1.335650e+04,1.990283,0.000,NaN


So the data im using breaks consumprion

In [27]:
tolist = DF.shape
DFlist = list(tolist)
print(DFlist)
rowsofdata = DFlist[0]
print(DFlist[0])

[55440, 11]


NameError: name 'DFlsit' is not defined

In [ ]:
#cleaning data as we only need the combind data

for row in rowsofdata:
    if row 